##Preparing Data

In [1]:
from modules import *
import sklearn

In [2]:
artists = pd.read_csv('data/artists.csv.gz', compression = 'gzip')
profiles = pd.read_csv('data/profiles.csv.gz', compression = 'gzip')
test = pd.read_csv('data/test.csv.gz', compression = 'gzip')
train = pd.read_csv('data/train.csv.gz', compression = 'gzip')

In [3]:
profiles.head()

,user,sex,age,country
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,f,25,Sweden
1,5909125332c108365a26ccf0ee62636eee08215c,m,29,Iceland
2,d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,m,30,United States
3,63268cce0d68127729890c1691f62d5be5abd87c,m,21,Germany
4,02871cd952d607ba69b64e2e107773012c708113,m,24,Netherlands


In [4]:
test.head()

,Id,user,artist
0,1,306e19cce2522fa2d39ff5dfc870992100ec22d2,4ac4e32b-bd18-402e-adad-ae00e72f8d85
1,2,9450d351278df4938bdea4ed86aec940a4e927ac,1f574ab1-a46d-4586-9331-f0ded23e0411
2,3,801909d6955f59033c88595d3d7f8a6a5dcd53cc,3eb72791-6322-466b-87d3-24d74901eb2d
3,4,e3ed47445c127fbeff47fb58f6bbf2f3b4535d82,61604b45-8a91-4e33-a1b6-45d7b1fec4e5
4,5,a73f46652103f3a5f7429159310f6928f79644aa,5dfdca28-9ddc-4853-933c-8bc97d87beec


In [5]:
data = pd.merge(left = train, right = profiles, how = 'left', on ='user')

In [4]:
gb = data.groupby('user')
a = gb.get_group('894484ed3c451c4468b043f363a4caf6de30f9ef')
b = gb.get_group('528200ac6ac9a35857d8952a26cf6f4738960643')
pd.merge(left = a, right = b, how = 'inner', on = 'artist', )

,user_x,artist,plays_x,sex_x,age_x,country_x,user_y,plays_y,sex_y,age_y,country_y
0,894484ed3c451c4468b043f363a4caf6de30f9ef,ef43cee7-13f5-4d10-844e-879aba3d8b19,16,f,16,United States,528200ac6ac9a35857d8952a26cf6f4738960643,42,f,16,United States
1,894484ed3c451c4468b043f363a4caf6de30f9ef,35a6a353-b186-4c13-a264-d18d5e2ce853,26,f,16,United States,528200ac6ac9a35857d8952a26cf6f4738960643,50,f,16,United States
2,894484ed3c451c4468b043f363a4caf6de30f9ef,a96ac800-bfcb-412a-8a63-0a98df600700,15,f,16,United States,528200ac6ac9a35857d8952a26cf6f4738960643,47,f,16,United States


In [5]:
demog = data.copy(deep = True)

In [6]:
demog = demog.drop('user', axis = 1)
demog['age'] = demog.age.replace(np.nan, 0) 
demog['sex'] = demog.sex.replace('NaN', -1) 
demog['sex'] = [1 if i == 'm' else (0 if i == 'f' else -1) for i in demog.sex]

In [7]:
testdf = pd.merge(left = test, right = profiles, how = 'left', on ='user')

In [8]:
testdf = testdf.drop('user', axis = 1)
testdf['age'] = testdf.age.replace(np.nan, 0) 
testdf['sex'] = testdf.sex.replace('NaN', -1) 
testdf['sex'] = [1 if i == 'm' else (0 if i == 'f' else -1) for i in testdf.sex]
testdf.head()

,Id,artist,sex,age,country
0,1,4ac4e32b-bd18-402e-adad-ae00e72f8d85,0,23,United Kingdom
1,125409,88a4cd4e-5dee-4153-8462-0d5e43c75f47,0,23,United Kingdom
2,463450,2e547c75-36c1-49d0-984e-b14498c936f0,0,23,United Kingdom
3,1793322,22a40b75-affc-4e69-8884-266d087e4751,0,23,United Kingdom
4,1953458,44cf61b8-5197-448a-b82b-cef6ee89fac5,0,23,United Kingdom


##min/diff

##General algorithm
1. For sex, age, country of user, find k-nearest neighbors that that have that country and that artist.
2. average plays of k-nearest neighbors.
3. weight with prior of average plays for that artist
4. return result

In [9]:
#clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None)
clf = sklearn.neighbors.NearestNeighbors()

In [10]:
prior = demog.groupby('artist').plays.mean()

In [ ]:
gb = demog.groupby(['artist','country'])

In [ ]:
t = datetime.now()
plays = []
for i, row in testdf.iterrows():
    result = 0
    try:
        index = tuple(row[['artist', 'country']])
        df = gb.get_group(index)
        df = df.reset_index()
        dist, indices = clf.fit(df[['sex','age']]).kneighbors(row[['sex', 'age']],n_neighbors = min(5, len(df)))
        result = df.ix[indices[0]].plays.mean() #Mean of k-nearest neighbors plays
        result = .2 * prior.ix[row['artist']] + .8*result #Incorporate prior belief
    except:
        #print (i, row)
        result = prior.ix[row['artist']]
    plays.append([i, result])
    if np.random.randint(10000) == 0:
        elapsed = datetime.now() - t
        total = (elapsed / i) * len(testdf)
        print "Time Remaining:", total - elapsed

Time Remaining: 3:55:25.434937
Time Remaining: 3:52:43.630694
Time Remaining: 3:49:11.280334
Time Remaining: 3:43:56.700864
Time Remaining: 3:43:52.189164
Time Remaining: 3:43:04.313674
Time Remaining: 3:42:52.556776
Time Remaining: 3:42:54.214208
Time Remaining: 3:42:48.579628
Time Remaining: 3:42:47.438788
Time Remaining: 3:42:16.407987
Time Remaining: 3:41:35.498311
Time Remaining: 3:40:44.485616
Time Remaining:

In [14]:
result = pd.DataFrame(plays)
result = result.apply(pd.Series.round)
result.columns = ['Id', 'Plays']
result.Id = result.Id + 1
result.to_csv('submission1.csv', index = False)

In [34]:
#Trying removing prior
bias = []
for i, row in testdf.iterrows():
    bias.append(.2 * prior.ix[row['artist']])

In [37]:
bias = pd.Series(bias)
result.Plays = (result.Plays - bias)*1.25

In [40]:
result = result.apply(pd.Series.round)
result.to_csv('submission2.csv', index = False)

In [41]:
prior.head()

artist
000d90ec-d64c-48a1-b775-e726fd240e9f    196.296078
000fc734-b7e1-4a01-92d1-f544261b43f5    288.887880
0019749d-ee29-4a5f-ab17-6bfa11deb969    145.452174
0039c7ae-e1a7-4a7d-9b49-0cbc716821a6    341.798469
004e5eed-e267-46ea-b504-54526f1f377d    370.565190
Name: plays, dtype: float64

In [42]:
testdf.head()

,Id,artist,sex,age,country
0,1,4ac4e32b-bd18-402e-adad-ae00e72f8d85,0,23,United Kingdom
1,125409,88a4cd4e-5dee-4153-8462-0d5e43c75f47,0,23,United Kingdom
2,463450,2e547c75-36c1-49d0-984e-b14498c936f0,0,23,United Kingdom
3,1793322,22a40b75-affc-4e69-8884-266d087e4751,0,23,United Kingdom
4,1953458,44cf61b8-5197-448a-b82b-cef6ee89fac5,0,23,United Kingdom


In [44]:
sub3 = testdf.artist.replace(prior)

In [46]:
sub3 = pd.DataFrame(sub3).reset_index()
sub3.columns = ['Id','Plays']
sub3['Id'] = sub3.Id + 1
sub3 = sub3.apply(pd.Series.round)
sub3.to_csv('submission3.csv', index = False)

##Create a BETTER input matrix for our analysis

In [6]:
gb = train.groupby('user')

In [7]:
z = np.zeros((len(train.user.unique()), len(train.artist.unique())))

In [8]:
X = pd.DataFrame(z)

In [9]:
X.index = train.user.unique()

In [10]:
X.columns = train.artist.unique()

In [11]:
for i,df in gb:
    for ind, row in df.iterrows():
        X[row.artist][row.user] = row.plays

In [12]:
X.head()

,5a8e07d5-d932-4484-a7f7-e700793a9c94,a3a92047-be1c-4f3e-8960-c4f8570984df,eeb1195b-f213-4ce1-b28c-8565211f8e43,a1419808-65d3-4d40-998c-1a0bac65eabc,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432,1cc5adcd-1422-4b5c-a3cd-3ecd4f43f506,9c9f1380-2516-4fc9-a3e6-f9f61941d090,832a43c7-aa7d-439b-a6b4-4f1afa671c24,2fddb92d-24b2-46a5-bf28-3aed46f4684c,847e8284-8582-4b0e-9c26-b042a4f49e57,...,e390d54f-92f0-4039-9dd3-e8eac130b4c0,72d7d717-0837-4f2a-9641-d0f9fdd3acf7,6ae51665-8261-4ae5-883f-1899651ad31b,ca264abf-3eb6-4d53-827b-6ab16988a4a3,c78a5d46-c631-41bc-8772-439002e3d3aa,dd3f655b-7bf6-4fe1-9360-3de5d842e9d2,582fee78-d934-4603-9f5c-1729e0ca36b7,c1d4f2ba-cf39-460c-9528-6b827d3417a1,bfcc6d75-a6a5-4bc6-8282-47aec8531818,774666d2-2064-4d6c-856c-f8cda0aaf9f0
eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03,554,0,0,0,0,554,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44ce793a6cd9d20f13f4a576a818ef983314bb5d,0,81,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
da9cf3f557161d54b76f24db64be9cc76db008e3,0,0,708,0,0,0,145,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8fa49ab25d425edcf05d44bfc1d5aea895287d81,0,0,0,265,0,0,0,0,476,0,...,0,0,0,0,0,0,0,0,0,0
b85fcaef67d2669cd99b334b5e8c8705263db2cf,0,0,0,0,220,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##Normalize rows of X so that number of plays doesn't affect user comparisons (however keep X) around and call this new normalized matrix Xnorm. If Xnorm doesn't yield good results, try X with min/diff similarity metric

In [13]:
Xnorm = X.div(X.sum(axis=1), axis=0)
Xnorm.head()

,5a8e07d5-d932-4484-a7f7-e700793a9c94,a3a92047-be1c-4f3e-8960-c4f8570984df,eeb1195b-f213-4ce1-b28c-8565211f8e43,a1419808-65d3-4d40-998c-1a0bac65eabc,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432,1cc5adcd-1422-4b5c-a3cd-3ecd4f43f506,9c9f1380-2516-4fc9-a3e6-f9f61941d090,832a43c7-aa7d-439b-a6b4-4f1afa671c24,2fddb92d-24b2-46a5-bf28-3aed46f4684c,847e8284-8582-4b0e-9c26-b042a4f49e57,...,e390d54f-92f0-4039-9dd3-e8eac130b4c0,72d7d717-0837-4f2a-9641-d0f9fdd3acf7,6ae51665-8261-4ae5-883f-1899651ad31b,ca264abf-3eb6-4d53-827b-6ab16988a4a3,c78a5d46-c631-41bc-8772-439002e3d3aa,dd3f655b-7bf6-4fe1-9360-3de5d842e9d2,582fee78-d934-4603-9f5c-1729e0ca36b7,c1d4f2ba-cf39-460c-9528-6b827d3417a1,bfcc6d75-a6a5-4bc6-8282-47aec8531818,774666d2-2064-4d6c-856c-f8cda0aaf9f0
eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03,0.034378,0.000000,0.000000,0.000000,0.000000,0.034378,0.000000,0,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
44ce793a6cd9d20f13f4a576a818ef983314bb5d,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
da9cf3f557161d54b76f24db64be9cc76db008e3,0.000000,0.000000,0.152883,0.000000,0.000000,0.000000,0.031311,0,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
8fa49ab25d425edcf05d44bfc1d5aea895287d81,0.000000,0.000000,0.000000,0.044329,0.000000,0.000000,0.000000,0,0.079625,0,...,0,0,0,0,0,0,0,0,0,0
b85fcaef67d2669cd99b334b5e8c8705263db2cf,0.000000,0.000000,0.000000,0.000000,0.141753,0.000000,0.000000,0,0.000000,0,...,0,0,0,0,0,0,0,0,0,0


##Run PCA to reduce dimensionality of our matrix

In [17]:
X.to_hdf('X.h5','df',mode='w')

ImportError: HDFStore requires PyTables

In [106]:
from sklearn.decomposition import PCA
pca = PCA(n_components='mle')

In [ ]:
pca.fit(Xnorm)